In [2]:
!pip install --upgrade pip
!pip install gensim

     |████████████████████████████████| 24.2 MB 179 kB/s eta 0:00:01
     |████████████████████████████████| 122 kB 82.5 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.2.1-py3-none-any.whl size=114113 sha256=e4e7d76b0c21f9bd45f4548a75e3c9925260ae153c75183ba10c2de6c2a6e764
  Stored in directory: /home/ec2-user/.cache/pip/wheels/12/6f/d1/6b06b392acb738f552f25dcfe0aac54a20ec854a20f55240aa
Successfully built smart-open
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [47]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import re
from src import utils, nmf_utils, nlp_topic_utils
from bs4 import BeautifulSoup

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import string

from gensim.corpora.dictionary import Dictionary
from gensim.models import Word2Vec

from collections import Counter
from operator import itemgetter
import itertools

from functools import reduce

In [50]:
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')

In [51]:
contracted_words_dict = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I would",
    "i'd've": "I would have",
    "i'll": "I will",
    "i'll've": "I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you you will",
    "you'll've": "you you will have",
    "you're": "you are",
    "you've": "you have"
}

In [52]:
addtl_stop_words = ['still', 'got', 'guy', 'say', 'still', 'thing', 'us', 'will', 'get', 'ok', 'we', 'please', 'would', 'could', 'im']

stop_words = stopwords.words("english")#.append(append_to_stopwords)
stop_words = stop_words + addtl_stop_words
stop_words_dict = Counter(stop_words)

# List of punctuation
addtl_punc = ['...']
punc = list(set(string.punctuation))
punc = punc + addtl_punc

In [87]:
print(punc)

['+', '-', '~', '%', '<', '"', ',', '*', '?', ')', ']', '&', '!', '(', '@', '|', '^', '[', ';', ':', '.', "'", '>', '/', '$', '#', '=', '\\', '{', '}', '`', '_', '...']


# Word2Vec

In [75]:
wallstreet = pd.read_csv('data/wallstreet_master.csv')

wallstreet['all_words_prep'] = wallstreet['processed_text'].apply(lambda x: nlp_topic_utils.process_text(x, contracted_words_dict, punc, stop_words_dict))
docs_raw = wallstreet['all_words_prep']

In [76]:
word2vec = Word2Vec(docs_raw, min_count=2)

In [79]:
vocab = word2vec.wv.vocab
print(type(vocab))

<class 'dict'>


In [81]:
# View "changelog" as a vector.
v1 = word2vec.wv['mdma']
print(v1)

[ 0.65311855  0.94201803  0.14269337 -0.5257336  -0.8555449  -0.46218452
  1.2190264  -2.4729989  -0.26393202 -0.6104909   0.33278066 -1.693457
 -0.18854004 -3.0759792  -0.5533608   2.689345   -2.8204706   1.0734991
  2.4886277  -0.1982572   0.23905     1.723087   -0.25764936 -0.43112382
 -0.50280887  1.5443712  -2.020272   -0.93054974 -0.7374248   0.40457746
  0.25046074 -0.48556313  1.6886201   0.52015805  1.4990027  -0.89978176
  1.4603829   1.8851442   0.38703313  1.0287033   2.0264008  -0.16804188
  0.7542263  -0.878981    2.1833062   1.8344841   1.3903927  -1.1597723
  0.1944612  -1.8686496   1.2394089  -0.15674637 -0.6241304   2.0034
 -1.8076298   1.0478396   0.7345203   1.6990949  -3.3628402  -0.20444939
  1.5147272  -1.5005238  -0.59681356  1.8977493  -1.1814281   0.7341676
 -1.4988227   2.4897432  -2.1666205  -0.6209147  -0.33160916 -1.6280396
 -0.05521302 -0.1004485   3.0875382   0.82738304 -1.9408144  -0.13068958
 -0.00381218  1.0902591  -1.5302418  -1.2752333  -0.33042186 

In [83]:
sim_words = word2vec.wv.most_similar('phished')
print(sim_words)

[('):', 0.7180207371711731), ('used.pm', 0.6576567888259888), ('woul', 0.6512549519538879), ('communal', 0.6490882635116577), ('pcno', 0.635083019733429), ('rule', 0.6334487199783325), ('sales.hey', 0.6287518739700317), ('account.message', 0.6265997290611267), ('trailstwl', 0.623162031173706), ('approving', 0.6213052272796631)]


In [86]:
sim_words = word2vec.wv.most_similar('arab')
print(sim_words)

[('sand', 0.9904016256332397), ('nun', 0.9833921194076538), ('eskimo', 0.9741783738136292), ('cryptorchidism', 0.9439956545829773), ('possible.strongly', 0.9161189198493958), ('malloc', 0.911230206489563), ('criminally', 0.910182535648346), ('violence', 0.9053794741630554), ('___quick', 0.9009281396865845), ('proclamation', 0.8985494375228882)]


In [88]:
sim_words = word2vec.wv.most_similar('btc')
print(sim_words)

[('bitcoin', 0.8482009768486023), ('bitcoins', 0.8365737199783325), ('fiat', 0.8198570013046265), ('xmr', 0.806617259979248), ('exchange', 0.7924795150756836), ('eth', 0.7710550427436829), ('volatility', 0.7611470818519592), ('coin', 0.7512288689613342), ('convert', 0.7490225434303284), ('monero', 0.7458834648132324)]
